In [1]:
import pandas as pd
import numpy as np
import time
import os
import glob
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
import string 
import helper_functions as hf

In [2]:
cwd=Path.cwd().resolve().parents[0]

In [3]:
BlogTO = pd.read_csv(os.path.join(cwd, "Data", "BlogTO_mentions.csv"))
print(BlogTO.shape)
BlogTO.head()

(47, 7)


,restaurant,category,stars,location,mention_count,latitude,longitude
0,Tiger Sugar,Bubble Tea,4.0,348A Yonge Street,1,43.658036,-79.381941
1,Chick-fil-A,Fast Food,4.5,709 Yonge Street,11,43.669547,-79.386144
2,Phat Kaphrao,Thai,4.5,3016B Bathurst Street,1,43.717842,-79.429744
3,Azkadenya - Mezza Diner,Middle Eastern,4.5,235 Queen Street West,0,43.650249,-79.388288
4,EAT BKK,Thai,4.5,580 Queen Street W,0,NaN,NaN


In [4]:
subway = pd.read_csv(os.path.join(cwd, "Data", "Torornto_Subway_Station_List.csv"))
print(subway.shape)
subway.head()

(142, 5)


,station_GO,line_number,Line_Color,latitude,longitude
0,Finch Station,1,Yellow,43.782083,-79.416066
1,North York Centre Station,1,Yellow,43.768595,-79.412732
2,Sheppard-Yonge Station,1,Yellow,43.761505,-79.410989
3,York Mills Station,1,Yellow,43.744069,-79.406720
4,Lawrence Station,1,Yellow,43.726597,-79.402592


In [5]:
radius=2 #km
restaurant_dict=dict()
for i in range(0, BlogTO.shape[0]):
    info_dict=dict()
    in_range_index=list()
    distances=hf.distance_func(BlogTO.loc[i,["latitude","longitude"]].tolist(),subway[["latitude","longitude"]].values.tolist())
    in_range_indice=[index for index,value in enumerate(distances) if value < radius]
    in_range_stations=subway.loc[in_range_indice,"station_GO"].tolist()
    info_dict["number_of_stations_({0}km)".format(radius)]=len(in_range_stations)
    info_dict["list_of_stations_({0}km)".format(radius)]='; '.join(in_range_stations)
    min_dist,min_case =hf.mindist(distances,subway["station_GO"].tolist())
    info_dict["closest_station"]=min_case
    info_dict["closest_station_km"]=min_dist
    restaurant_dict[BlogTO.loc[i,"restaurant"]]=info_dict

In [6]:
restaurant_info_df=pd.DataFrame(restaurant_dict).T  
restaurant_info_df=restaurant_info_df.reset_index().rename(columns={"index":'restaurant'})
final_BlogTO=pd.merge(BlogTO, restaurant_info_df, on = 'restaurant')

In [7]:
final_BlogTO.sort_values(by=[c for c in final_BlogTO.columns if "number_of_stations" in c][0], ascending =False)

,restaurant,category,stars,location,mention_count,latitude,longitude,number_of_stations_(2km),list_of_stations_(2km),closest_station,closest_station_km
44,La Diperie,Ice Cream & Frozen Yogurt,3.5,717 Bay St,0,43.659556,-79.384837,18,Rosedale Station; Wellesley Station; College S...,College Station,0.213561
21,Boat King of Thai Noodles,Thai,3.0,770 Bay Street,1,43.660101,-79.386004,18,Rosedale Station; Wellesley Station; College S...,College Station,0.264928
16,The Alley,Bubble Tea,5.0,382 Yonge Street,1,43.659289,-79.382993,17,Rosedale Station; Wellesley Station; College S...,College Station,0.174151
7,Xing Fu Tang,Bubble Tea,3.5,506 Yonge Street,0,43.663343,-79.384151,17,Rosedale Station; Wellesley Station; College S...,Wellesley Station,0.233564
11,Bloom,Cafes,4.0,596 Yonge street,2,43.665505,-79.385056,17,Summerhill Station; Rosedale Station; Wellesle...,Wellesley Station,0.108039
18,TRU TEA 初茶,Bubble Tea,4.0,25 Carlton Street,0,43.661088,-79.382001,17,Rosedale Station; Wellesley Station; College S...,College Station,0.0753076
14,Avelo,Vegan,4.5,51 St Nicholas Street,1,43.667066,-79.386168,17,Summerhill Station; Rosedale Station; Wellesle...,Wellesley Station,0.268951
45,Ronin Izakaya,Japanese,3.5,39 Baldwin Street,0,43.655827,-79.393590,16,Wellesley Station; College Station; Queen Stat...,St. Patrick Station,0.453297
39,Higher,Bubble Tea,3.5,493A Dundas St W,0,43.652864,-79.397544,16,Wellesley Station; College Station; Queen Stat...,St. Patrick Station,0.775935
35,Project Glyph,Desserts,5.0,40 Hayden Street,2,43.669967,-79.384594,16,Summerhill Station; Rosedale Station; Wellesle...,Wellesley Station,0.51049


In [8]:
final_BlogTO.to_csv(os.path.join(cwd, "Data", "final_BlogTO.csv"),index=False)